<a href="https://colab.research.google.com/github/anespart1/educationuse/blob/main/Datavisualization_of_weather_data(simple).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#셀레니움 설치 / 가져오기 / 크롬드라이버 준비

In [26]:
#처음 실행시 아래 3항목 #제거 필요
#!pip install selenium
#!apt-get update
#!apt install chromium-chromedriver

from selenium import webdriver
from selenium.webdriver.support.ui import Select
import matplotlib.pyplot as plt
import pandas as pd
import plotly.offline
import plotly.express as px
import plotly.io as pio

#옵션 설정 및 크롬 드라이버 준비
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

#플로티 옵션 설정 및 준비
plotly.offline.init_notebook_mode(connected=True)
pio.renderers.default = 'colab'

# 기상청 데이터 시각화(심플버전)

In [2]:
#@title 지역을 선택해주세요 { run: "auto" }
option1 = "\uBC31\uB839\uB3C4(\uC720)" #@param ['서울(유)','백령도(유)','동두천(무)','파주(무)','인천(유)','수원(유)','강화(무)','양평(무)','이천(무)','북춘천(유)','철원(무)','춘천(무)','원주(무)','영월(무)','인제(무)','홍천(무)','북강릉(유)','울릉도(유)','강릉(무)','속초(무)','대관령(무)','동해(무)','태백(무)','정선군(무)','청주(유)','충주(무)','추풍령(무)','제천(무)','보은(무)','홍성(유)','대전(유)','서산(무)','천안(무)','보령(무)','부여(무)','금산(무)','전주(유)','군산(무)','부안(무)','임실(무)','정읍(무)','남원(무)','장수(무)','순창(무)','고창(무)','고창(구)','광주(유)','목포(유)','흑산도(유)','여수(유)','완도(무)','진도(첨찰산)(무)','진도군(무)','영광(무)','순천(무)','순천(구)','장흥(무)','해남(무)','고흥(무)','강진군(무)','보성군(무)','광양(무)','안동(유)','포항(유)','대구(유)','대구(구)','울진(무)','상주(무)','봉화(무)','영주(무)','문경(무)','영덕(무)','의성(무)','구미(무)','영천(무)','청송군(무)','경주(무)','부산(유)','울산(유)','창원(유)','북창원(무)','통영(무)','진주(무)','거창(무)','합천(무)','밀양(무)','산청(무)','거제(무)','남해(무)','김해시(무)','양산(무)','의령군(무)','함양군(무)','제주(유)','고산(무)','서귀포(무)','성산(무)']
print('{} 를 선택했습니다.'.format(option1))

백령도(유) 를 선택했습니다.


In [3]:
#@title 연도를 선택해주세요. { run: "auto" }
option2 = 2012  #@param {type: "slider", min: 1960, max: 2021}
print('{} 를 선택했습니다.'.format(option2))

2012 를 선택했습니다.


In [4]:
#@title 데이터를 선택해주세요 { run: "auto" }
option3 = "\uD3C9\uADE0\uAE30\uC628" #@param ['평균기온', '최저기온', '최고기온', '강수량', '신적설', '평균풍속', '상대습도', '일조시간', '운량', '날씨']
print('{} 를 선택했습니다.'.format(option3))

평균기온 를 선택했습니다.


## 코드 실행부(숨김)

In [78]:
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
driver.get('http://www.weather.go.kr/weather/climate/past_table.jsp') #기상청 날씨누리 데이터 가져오기

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [79]:
#옵션값 가져오는 방법(수정용)
#menu = driver.find_element_by_id('observation_select3').text
#menu = menu.split('\n')
#am = [i.lstrip() for i in menu]
#bm = [i for i in am if '-' not in i and len(i) > 1]
#print(bm)

In [80]:
select = Select(driver.find_element_by_id('observation_select1'))
select.select_by_visible_text(option1)
driver.find_element_by_xpath('//*[@id="content_weather"]/form/fieldset/input[1]').click()

In [81]:
select = Select(driver.find_element_by_id('observation_select2'))
select.select_by_visible_text(str(option2))
driver.find_element_by_xpath('//*[@id="content_weather"]/form/fieldset/input[2]').click()

In [82]:
select = Select(driver.find_element_by_id('observation_select3'))
select.select_by_visible_text(option3)
driver.find_element_by_xpath('//*[@id="content_weather"]/form/fieldset/input[3]').click()

In [83]:
table = driver.find_element_by_xpath('//*[@id="content_weather"]/table')

#테이블 열이름 가져오기
heads = table.find_elements_by_class_name('top_line')
thead = [value.text for index, value in enumerate(heads)]

#테이블 각 행값 가져오기
tbody = table.find_element_by_tag_name("tbody")
rows = tbody.find_elements_by_tag_name("tr")
trows = []
for a_row in rows:
  body = a_row.find_elements_by_tag_name("td")
  trow = []
  for index, value in enumerate(body):
    trow.append(value.text)
  trows.append(trow)

In [84]:
df = pd.DataFrame(data=trows, columns=thead)
#print('[ {}(℃) ] {} / {}년'.format(option1, option2, option3))
#df

## 월별 기온 표시 부분

In [118]:
#@title 달(月)을 선택해주세요 { run: "auto" }
option4 = "1\uC6D4" #@param ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월']

df1 = df[df[option4]!=' '].copy()
df2 = df1[option4]
df2 = pd.to_numeric(df2) #숫자형 데이터로 변경
df2 = df2.reset_index()
df2['index'] = df2['index'] + 1
df2.columns = ['날짜', '기온']

title = '{}년 {} {} {} 그래프'.format(option2, option4, option1.split('(')[0], option3)
fig = px.line(df2, x="날짜", y="기온", title=title)
fig.show()